# Tarea 9

En esta tarea busco predecir la proporción de propina que dan los usuarios de taxi en sus viajes. Para esto se aprovecha el computo en paralelo que ofrece Dask.

* Leo los datos

In [1]:
import pandas as pd
trips_df = pd.read_csv("/data/trips.csv")
trips_df.head()

,car_type,fare_amount,passenger_count,taxi_id,tip_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance
0,A,22.0,1,1,4.60,2015-01-03 01:37:02,2015-01-03 01:17:32,6.90
1,A,9.0,1,1,0.00,2015-01-05 23:35:02,2015-01-05 23:25:15,1.81
2,A,7.5,1,1,1.00,2015-01-06 15:22:12,2015-01-06 15:11:45,0.96
3,A,8.5,1,1,1.00,2015-01-08 08:31:23,2015-01-08 08:22:12,1.90
4,A,7.5,1,1,1.66,2015-01-08 12:35:54,2015-01-08 12:26:26,1.00


* obtengo el porcentaje de propina y el logaritmo del costo del viaje

In [2]:
import numpy as np

trips_df['tip_perc'] = trips_df['tip_amount'] / trips_df['fare_amount']
trips_df['log_fare_amount'] = np.log(trips_df['fare_amount'])

/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in log
  after removing the cwd from sys.path.


In [3]:
trips_df = trips_df.dropna()

## Ejercicio 1

### ¿Qué tan rápido es buscar en paralelo comparado con una búsqueda secuencial en python?

* preparo los datos para usar Ridge()

In [4]:
Y_1 = trips_df['tip_perc'] 
X_1 = trips_df[['car_type', 'fare_amount', 'log_fare_amount']]
X_1['car_type'] =  X_1['car_type'].astype('category')

/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


* convierto a dummy `car_type`

In [5]:
car_dummies = pd.get_dummies(X_1['car_type'], prefix='car_type')
X_1 = pd.concat([X_1, car_dummies], axis=1)
X_1.drop('car_type', axis=1, inplace=True)

In [6]:
from sklearn.linear_model import Ridge

lr = Ridge()
#lr.fit(X_1, Y_1)
#lr.predict(X_1)

* preparo el GridSearch (modifico nivel de regularización)

In [7]:
parametros = {'alpha': [0.001, 0.1, 1, 10, 1000]}

* buscamos el modelo con los mejores hiperparametros

In [8]:
from sklearn.grid_search import GridSearchCV

busqueda_seq = GridSearchCV(estimator = Ridge(), 
                            param_grid = parametros, 
                            scoring='neg_mean_squared_error',  
                            cv=3)
busqueda_paralell = GridSearchCV(estimator = Ridge(),
                                 param_grid = parametros, 
                                 scoring='neg_mean_squared_error',
                                 cv=3)
from dask import delayed

def run_grid(X, Y):
    return busqueda_paralell.fit(X, Y)

delayed_run_grid = delayed(run_grid)

/usr/local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.5/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [9]:
print('Sequencial:')
%time busqueda_seq.fit(X_1, Y_1)
print('En paralelo:')
%time run_grid(X_1, Y_1)

Sequencial:
CPU times: user 250 ms, sys: 290 ms, total: 540 ms
Wall time: 159 ms
En paralelo:
CPU times: user 180 ms, sys: 330 ms, total: 510 ms
Wall time: 137 ms


GridSearchCV(cv=3, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': [0.001, 0.1, 1, 10, 1000]},
       pre_dispatch='2*n_jobs', refit=True,
       scoring='neg_mean_squared_error', verbose=0)

* mejores scores

In [10]:
print(busqueda_seq.best_score_)
print(busqueda_paralell.best_score_)

-0.016428218559979186
-0.016428218559979186


* mejores parametros

In [11]:
print(busqueda_seq.best_estimator_)
print(busqueda_paralell.best_estimator_)

Ridge(alpha=1000, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)
Ridge(alpha=1000, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)


## Ejercicio 2

### ¿Cómo se comparan los tiempos de ejecución de tu búsqueda con la de Dask ML?

* convertierto a dummy `car_type`

In [12]:
Y_2 = trips_df['tip_perc'] 
X_2 = trips_df[['car_type', 'fare_amount', 'log_fare_amount']]
X_2['car_type'] =  X_2['car_type'].astype('category')

/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
from dask_ml.preprocessing import DummyEncoder

codificador = DummyEncoder()
X_2 = codificador.fit_transform(X_2)

* preparo los datos para usar `LinearRegression()`

In [14]:
import dask.array as da

Y_2 = da.from_array(np.asarray(Y_2), chunks=(3))
X_2 = da.from_array(np.asarray(X_2), chunks=(3, 3))

#http://dask.pydata.org/en/latest/array-creation.html#unknown-chunks

In [15]:
from dask_ml.linear_model import LinearRegression

lr = LinearRegression()
#lr.fit(X_2, Y_2)
#lr.predict(X_2).compute()

* preparamos el GridSearch

In [16]:
from dask_ml.model_selection import GridSearchCV
#http://dask-searchcv.readthedocs.io/en/latest/

In [17]:
parameters = {'C': [0.001, 0.1, 1, 10, 1000]}

* buscamos el mejor modelo

In [18]:
busqueda = GridSearchCV(lr, parameters)
% time busqueda.fit(X_2, Y_2)

CPU times: user 1min 1s, sys: 2min 7s, total: 3min 9s
Wall time: 58.2 s


GridSearchCV(cache_cv=True, cv=None, error_score='raise',
       estimator=LinearRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
         intercept_scaling=1.0, max_iter=100, multiclass='ovr', n_jobs=1,
         penalty='l2', random_state=None, solver='admm',
         solver_kwargs=None, tol=0.0001, verbose=0, warm_start=False),
       iid=True, n_jobs=-1, param_grid={'C': [0.001, 0.1, 1, 10, 1000]},
       refit=True, return_train_score='warn', scheduler=None, scoring=None)

In [19]:
busqueda.best_score_

0.050288570564370964

In [20]:
busqueda.best_params_

{'C': 0.1}

## Conclusiones

* En este caso, el mejor método es el secuencial.
* En todos los casos (sklearn secuencial, sklearn paralelo y  dask_ml) el mejor parámetro de regularización es 1000.
* Por alguna razón la implementación de Ridge (sklearn) y LinearRegression (dask_ml) regresan evaluaciones distintas aunque se utilizan las mismas variables, cv-folds, hiperparametros
